^C


In [ ]:

from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from typing import List, Dict
from langchain_together import ChatTogethe

ImportError: cannot import name 'AGENT_DEPRECATION_WARNING' from 'langchain._api.deprecation' (c:\Users\Manu\anaconda3\Lib\site-packages\langchain\_api\deprecation.py)

In [ ]:
# Initialize the LLM
llm = ChatTogether(
        api_key="5081cda228b7435ec59fb6a8ca3c40044d4b3a6c3941aa7e479add4fdd489984",
        model="google/gemma-2-27b-it",
    )

# Step 1: Quiz Generation Tool
class QuizGeneratorTool(BaseTool):
    name = "Quiz Generator"
    description = "Generates a quiz based on a given topic and number of questions."

    def _run(self, topic: str, num_questions: int = 5) -> List[Dict]:
        prompt = PromptTemplate(
            input_variables=["topic", "num_questions"],
            template="Generate {num_questions} quiz questions about {topic}. Each question should have 4 options and a correct answer. Format the output as a list of dictionaries with keys 'question', 'options', and 'answer'."
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        quiz = chain.run(topic=topic, num_questions=num_questions)
        return eval(quiz)  # Convert the string output to a list of dictionaries

    async def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported")

# Step 2: Quiz Evaluation Tool
class QuizEvaluatorTool(BaseTool):
    name = "Quiz Evaluator"
    description = "Evaluates the user's answers to the quiz and provides feedback."

    def _run(self, quiz: List[Dict], user_answers: List[int]) -> Dict:
        score = 0
        feedback = []
        for i, q in enumerate(quiz):
            if user_answers[i] == q['options'].index(q['answer']) + 1:
                score += 1
                feedback.append(f"Question {i+1}: Correct! 🎉")
            else:
                feedback.append(f"Question {i+1}: Incorrect. The correct answer was: {q['answer']}")
        return {"score": score, "feedback": feedback}

    async def _arun(self, *args, **kwargs):
        raise NotImplementedError("Async not supported")

# Step 3: Initialize the Agent
tools = [QuizGeneratorTool(), QuizEvaluatorTool()]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Step 4: Main Function
def main():
    print("Welcome to the Agentic Quiz Generator!")
    topic = input("Enter a topic you're interested in: ")
    num_questions = int(input("How many questions would you like? "))

    # Generate the quiz
    quiz = agent.run(f"Generate a quiz about {topic} with {num_questions} questions using the Quiz Generator tool.")
    print("\nQuiz Generated! Let's begin.\n")

    # Take the quiz
    user_answers = []
    for i, q in enumerate(quiz):
        print(f"Question {i+1}: {q['question']}")
        for j, option in enumerate(q['options']):
            print(f"{j+1}. {option}")
        answer = int(input("Your answer (1-4): "))
        user_answers.append(answer)

    # Evaluate the quiz
    result = agent.run(f"Evaluate the quiz with the following user answers: {user_answers} using the Quiz Evaluator tool.")
    print("\nQuiz Results:")
    print(f"Your score: {result['score']}/{len(quiz)}")
    for feedback in result['feedback']:
        print(feedback)